In [2]:
import os
from openai import OpenAI

In [3]:
# Get API key from environment variable
client = OpenAI(api_key="API_KEY")  # Replace with your full API key

In [4]:
def read_file_content(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

In [517]:
def DMN_Guided_Prompting(dmn_content, case_text):
  prompt = """
You are given three inputs:

1. A DMN file in XML format. It may contain one or more decision tables.  
2. A natural language description of a customer case.  
3. (Optional) An example input illustrating what such descriptions typically look like.  

Your task is to extract the decision logic from the DMN and reason through the customer case using the following steps.

---

### Phase 1: Extract Decision Logic from the DMN

From the DMN file:
- Identify each decision rule (preferably using annotations as rule names).
- For each rule, extract a structured dictionary entry as follows:

```json
{
  "Rule Name": {
    "input": [list of required input field names],
    "Decision table": {
      "name": "<decision table name>",
      "content": "<decision table XML snippet>"
    },
    "Literal expression": {
      "name": "<output name>",
      "content": "<list of outcomes and corresponding messages>"
    }
  }
}

---

### Phase 2: Process Each Entry Using Natural Language Input

For each extracted entry:

**Step 1: Extract Input Values**
- Identify the required input fields from the "input" list.
- From the natural language text, extract the values for those fields (even if not in key-value format).

**Step 2: Evaluate the Decision Table**
- Evaluate input values strictly according to the decision table logic, any values shall be treated accordingly (numbers below are examples):
    - [40..50] means inclusive of 40 and 50.
    - (40..50) means exclusive of both 40 and 50.
    - ]40..50[ means exclusive of both 40 and 50.
    - [40..50) means inclusive of 40, exclusive of 50 (and so on).
    - [40..50[ means inclusive of 40, exclusive of 50 (and so on).  
    - Do not round or approximate values. For instance, 39 must not match [40..50].
- Identify the outcome name for the matching rule.

**Step 3: Retrieve the Corresponding Literal Expression**
- Use the outcome name concatenate it with '_message' and look at the message instruction written in the `"Literal expression"` entry.

**Step 4: Generate the Replies**
- return the message exactly as written, without any change.
- generate a list named **result** include all generated messages, i.e., "result": [

---

### Output Format
return only the result list.


"""

  response = client.chat.completions.create(
      model="gpt-4o",
      messages=[
              {"role": "system", "content": "You are a DMN expert."},
              {"role": "user", "content": prompt + 
                  "\n\n--- DMN FILE START ---\n" + dmn_content + "\n--- DMN FILE END ---" + 
                  "\n\n--- natural language text START ---\n" + case_text + "\n--- natural language text END ---"
              }
          ],
      temperature=0,
  )
  return response.choices[0].message.content

In [518]:
# Define project root
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), '..', '..'))

# # read dmn file and loan_approval_test1 from project root
dmn_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'process_feedback', 'loan_approval.dmn'))
txt_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'feedback_example', 'loan_approval_test1.txt'))

rule1_info = DMN_Guided_Prompting(dmn_content,txt_content)
print(rule1_info)

```json
[
    "We’d like to inform you that your application has been processed and requires a manual review. We will update you as soon as the review is complete."
]
```


In [519]:
# read dmn file and loan_approval_test2 from project root
dmn_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'process_feedback', 'loan_approval.dmn'))
txt_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'feedback_example', 'loan_approval_test2.txt'))

rule1_info = DMN_Guided_Prompting(dmn_content,txt_content)
print(rule1_info)

```json
[
  "We're pleased to inform you that your loan application has been approved. Congratulations!"
]
```


In [520]:
# # read dmn file and loan_approval_test3 from project root
dmn_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'process_feedback', 'loan_approval.dmn'))
txt_content = read_file_content(os.path.join(PROJECT_ROOT, 'src', 'feedback_example', 'loan_approval_test3.txt'))

rule1_info = DMN_Guided_Prompting(dmn_content,txt_content)
print(rule1_info)

```json
[
    "We regret to inform you that your loan application has not been approved due to either insufficient income or credit score. If you have any questions or would like further clarification, please feel free to contact us."
]
```
